In [ ]:
%run "catalog_common.py" 
ID_header('Open-FF: State Index', incl_links=True)
set_page_param()


In [ ]:
import pandas as pd
import numpy as np
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)
from itables import show as iShow
import itables.options as opt

df = ana_set.Full_set(repo = repo_name, outdir='../common/').get_set(verbose=False)
# gb2 = df_cas.groupby('bgCAS')['IngredientName'].agg(lambda x: x.value_counts().index[0])
# gb2 = gb2.reset_index()
# gb2.columns = ['bgCAS','comm_name']
# df_cas = pd.merge(df_cas,gb2,on='bgCAS',how='left')
# # df_cas.columns

In [ ]:
gb = df[(df.in_std_filtered)&(df.loc_within_state=='YES')].groupby(['bgStateName','bgCountyName','UploadKey'],as_index=False)['date'].first()
gb1 = gb.groupby('bgStateName',as_index=False)['UploadKey'].count().rename({'UploadKey':'Number of disclosures'},axis=1)

gb2 = gb.groupby(['bgStateName','bgCountyName'],as_index=False)['UploadKey'].count()
gb2 = gb2.groupby('bgStateName',as_index=False)['bgCountyName'].count().rename({'bgCountyName':'Number of counties'},axis=1)

gb3 = gb.groupby('bgStateName',as_index=False)['date'].agg(['min','max']).rename({'min':'earliest',
                                                                                  'max':'latest'},axis=1)
mg = pd.merge(gb1,gb2,on='bgStateName',how='left')
mg = pd.merge(mg,gb3,on='bgStateName',how='left')
mg = mg.rename({'bgStateName':'State'},axis=1)
# mg = mg.reset_index(drop=True)
mg.State = '<center><h3>'+mg.State.str.title().map(lambda x: getStateLink(x,x))+'</h3></center>'
mg = mg.sort_values('Number of disclosures',ascending=False)

import itables.options as opt
opt.order = []  # no sorting

iShow(mg.reset_index(drop=True),paging=False)